# Process ROH Results into one big dataframe
Contains cleaning lines (i.e. to remove duplicates), fix flipped coordinates

In [4]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colorbar as clb
import matplotlib.colors as cls
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

import socket
import os as os
import sys as sys
import multiprocessing as mp

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

### Additional Imports from Support Packages
sys.path.append("./package/hapsburg/")
from PackagesSupport.pp_individual_roh_csvs import extract_sub_df_geo_kw, give_df_clsts

midway2-0405.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


### Functions that pre-process Data
Add "region" Field. Add "color" (based on Time) field

In [28]:
############################################################################
### Post-Process Regions
def set_regions_from_csv(df, csv_path = "./Data/RegionDefinition/regions.csv", 
                         output=True, sep=","):
    """Set Region column in df, by loading coordinates from csv_path"""
    df_regions= pd.read_csv(csv_path, sep=sep)
    for index, row in df_regions.iterrows():
        region = row["Region"] 
        if output:
            print(f"Doing {region}...")
        kw = str(row["Keywords"]).split("|") # produce list from Keywords
        df_t = extract_sub_df_geo_kw(df, row["Lat_low"], row["Lat_high"], row["Lon_low"], 
                                     row["Lon_high"], kw, output=output)
        idx = df["iid"].isin(df_t["iid"]) # Get Indices of Sub Dataframe
        df.loc[idx, "region"] = row["Region"] 
    return df

############################################################################
### Post-Process Colors
def set_colors_from_csv(df, csv_path = "./Data/RegionDefinition/colors.csv", 
                         output=True, sep=","):
    """Set Color column in df, by loading colors from csv_path"""
    df_colors= pd.read_csv(csv_path, sep=sep)
    for index, row in df_colors.iterrows():
        color = row["Color"] 
        ig = row["InternalGroup"]
        kw = str(row["Keywords"]).split("|") # produce list from Keywords
        df_t = give_df_clsts(df, search=kw, col="pop")
        idx = df["iid"].isin(df_t["iid"]) # Get Indices of Sub Dataframe
        df.loc[idx, "color"] = color
        df.loc[idx, "clst"] = row["clst"]
        
        if output:
            print(f"Doing {ig}...")
            print(f"Found {np.sum(idx)} Inds - set to color: {color}")
    ### Do old HunterGatherers
    return df

def set_color_hg_minage(df, color="blue", min_age=10500, output=True):
    """Set the color for all ancient Huntergatherers."""
    idx = df["age"] > min_age
    df.loc[idx, "color"] = color
    if output:
        print(f"Found {np.sum(idx)} Inds >{min_age} BP - set to color: {color}")
    return df
    
def set_color_modern(df, color="white", output=True):
    """Set color for all Modern Samples"""
    idx = df["age"] == 0
    df.loc[idx, "color"] = color
    df.loc[idx, "clst"] = "Modern"
    if output:
        print(f"Found {np.sum(idx)} Moderns - set to color: {color}")
    return df

def remove_ids(df, csv_path = "./Data/RegionDefinition/remove_ids.csv", output=True, del_col="iid"):
    """Remove Individuals whose del_col column contains
    string from del_strings (list)"""
    del_list = np.loadtxt(csv_path, dtype="str", ndmin=1)
    
    n=len(df)
    for ds in del_list:
        idx =  df[del_col].str.contains(ds)
        #iids = df[del_col][idx].values
        #print(f"Deleting: {iids}")
        df = df[~idx]
    if output:
        print(f"Removed {n-len(df)} / {n} Individuals.")
    return df

def remove_duplicates(df, cov_col="n_cov_snp", id_col="iid", master_col = "Master ID",
                      path_master="./Data/ReichLabEigenstrat/Raw.v42.4/v42.4.1240K.anno",
                      output=True):
    """Remove duplicates based on merging with Master Dataframe.
    Return Filtered Dataframe
    id_col: Column onto which to merge
    """
    n = len(df)
    df_meta = pd.read_csv(path_master, sep="\t")
    df_meta[id_col] = df_meta.filter(regex='Instance ID')
    
    df_meta = df_meta[[id_col, master_col]]  # Only relevant columns
    df_merge = pd.merge(df, df_meta, on=id_col, how="left")  # Merge on IID
    df_merge = df_merge.sort_values(by=cov_col, ascending=False) # Put IIDs with most SNPs first
    ### Fill up NaNs with IDs
    idx = df_merge[master_col].isnull()
    df_merge.loc[idx, master_col] = df_merge.loc[idx, id_col]
    df_merge = df_merge.drop_duplicates(subset=master_col, keep="first")
    
    df_merge = df_merge.drop(columns=master_col)  #Drop the Master ID Col again

    if output:
        print(f"Removed {n- len(df_merge)} / {n} Duplicates")
    return df_merge

############################################################################
############################################################################

def merge_in_economy_iid(df, path_economy="", min_cert=3,
                         economy_col="economy", cty_col="certainty",
                         match_col = "iid", case=False):
    """Create/Set Column economy_col into dataframe df. Check for substring matches (to be future proof)
    Return modified dataframe.
    match_col: What columns to match
    economy_col: What column to transfer over
    case: Whether IID substring matching is case sensitive.
    min_cert: Minimum certainty"""
    df_match = pd.read_csv(path_economy)  # Load the data
    idx_uncertain = (df_match[cty_col] < min_cert)
    df_match.loc[idx_uncertain, economy_col] = "Low Certainty"
    
    ### Create new, empty economy column if needed
    if not economy_col in df.columns:
        df[economy_col] = np.nan
    
    ### Match all IIDs
    for i,m in enumerate(df_match[match_col]):
        m = m.rstrip()  # Remove all right whitespaces
        if len(m)==0:
            continue
        idx = df[match_col].str.contains(m, case=case)
        df.loc[idx, economy_col] = df_match.loc[i,economy_col]
    return df

def change_economy(df, col="economy",
                   str1="mixed", new="Mixed", output=True):
    """Change Economy assignment from str1 to new"""
    idx = (df[col].str.contains(str1)) & (~df_t[col].isnull())
    df.loc[idx, col] = new
    
    if output:
        print(f"Changing {np.sum(idx)} Individuals to: {new}")
    return df

def set_economy_color(df, path_color_df="./Data/RegionDefinition/economy_colors.csv", 
                      color_col="color", economy_col="economy"):
    """Set Color Based on Economy.
    Assume color column in df exists"""
    df_c = pd.read_csv(path_color_df)
    dct = dict(zip(df_c[economy_col], df_c[color_col]))  # Create mapping dictionary
    df[color_col] = df[economy_col].map(dct).fillna(df[color_col])  # Only Map hits
    return df

def set_latlon_from_csv(df, csv_path = "./Data/Coordinates/geo_clst.csv", 
                        output=True, sep="\t", col="pop", col_match="clst"):
    """Set Color column in df, by loading colors from csv_path"""
    df_geo = pd.read_csv(csv_path, sep=sep)

    for index, row in df_geo.iterrows():
        lat = row["lat"] 
        lon = row["lon"]
        kw = row[col_match]
        df_t = df[df[col].str.contains(kw)]
        idx = df["iid"].isin(df_t["iid"]) # Get Indices of Sub Dataframe
        df.loc[idx, "lat"] = lat
        df.loc[idx, "lon"] = lon

        if output:
            print(f"Doing {kw}...")
            print(f"Found {np.sum(idx)} Inds - set GPS")
    ### Do old HunterGatherers
    return df

def set_islands(df, df_isl, col="region", max_d=2, new_val="Islands", output=True):
    """Set col in dataframe df that are close to df_isl lat lon
    to new new_val. Return modified dataframe
    max_d: Maximum Distance (in latlon space)"""
    for index, row in df_isl.iterrows():
        lat_close = (np.abs(df["lat"] - row["lat"]) < max_d)
        lon_close = (np.abs(df["lon"] - row["lon"]) < max_d)
        idx = (lat_close & lon_close)
        
        if output:
            region1 = row["clst"]
            print(f"{region1}: {np.sum(idx)}")
        df.loc[idx, col] = new_val
    return df

# Load all varying Dataframes

In [29]:
### Reich Dataframe
# Define Individuals we want to delete (Duplicates/Neanderthals)
df_r = pd.read_csv("./Empirical/Eigenstrat/Reichall/combined_roh_v42.csv", sep="\t")
df_r['region'] = "all"   # Place Holder
print(f"Loaded Reich Data: {len(df_r)}")
cols = df_r.columns # Extract key column names in right order

### Sardinians from Marcus et all
df_sard = pd.read_csv("./Empirical/1240k/MarcusAncs/combined_roh05.csv", sep="\t")
df_sard = df_sard[df_sard["pop"].str.contains("Sar-")]  #Extract Sardinia Data
df_sard["region"]="Sardinia"
df_sard = df_sard[cols]
print(f"Loaded Sardinian Data: {len(df_sard)}")

### Human Origin Data
df_ho = pd.read_csv("./Empirical/HO/combined_roh05.csv", sep="\t")
df_ho["region"] = df_ho["pop"] # Will be later overwritten for Macro Region!
df_ho["color"] = "gray"
df_ho = df_ho[cols]
print(f"Loaded modern Data: {len(df_ho)} Individuals")

### Concatenate the Dataframes
df_all = pd.concat([df_r, df_sard, df_ho])
print(f"Concatenated {len(df_all)} Individual ROH Data!")

### Filter to good individuals
df_all =df_all[df_all["include_alt"]>0] 
print(f"Filtered to {len(df_all)} Individuals with include_alt>0")

Loaded Reich Data: 1923
Loaded Sardinian Data: 40
Loaded modern Data: 1941 Individuals
Concatenated 3904 Individual ROH Data!
Filtered to 3811 Individuals with include_alt>0


### Remove Individuals in Deletion List and also Duplicates 
(based on master ID)

In [30]:
df_all = remove_ids(df_all, csv_path="./Data/RegionDefinition/remove_study.csv", del_col="study")
df_all = remove_ids(df_all, csv_path="./Data/RegionDefinition/remove_ids.csv", del_col="iid")
df_all = remove_duplicates(df_all, path_master="./Data/ReichLabEigenstrat/Raw.v42.4/v42.4.1240K.anno")

Removed 12 / 3811 Individuals.
Removed 15 / 3799 Individuals.
Removed 58 / 3784 Duplicates


### Merge in Coordinats

### Fill in missing coordinates from outside source

In [32]:
df_geo = pd.read_csv("./Data/Coordinates/MittnikNatComm2018_Coordinates.csv", sep="\t")
df_geo.index = df_geo["iid"]
df_all.index = df_all["iid"]
df_all.update(df_geo)

### Merge in group GPS
df_all = set_latlon_from_csv(df_all, csv_path = "./Data/RegionDefinition/geo_clst.csv", col="clst",
                    output=False, sep="\t")

### Merge in individuals GPS
df_all = set_latlon_from_csv(df_all, csv_path = "./Data/RegionDefinition/geo_iid.csv", 
                             col="iid", col_match="iid", output=False, sep="\t")

### Set the regions from .csv
csv_path = "./Data/RegionDefinition/regions.csv"
df_t = set_regions_from_csv(df_all, csv_path)

### Set the Islands
df_isl = pd.read_csv("./Data/RegionDefinition/islands_gps.csv", sep="\t")
df_t = set_islands(df_t, df_isl, col="region", new_val="Islands")

Doing Iberia...
Found 244 Individuals; 214 from Geography
Doing Balkans...
Found 149 Individuals; 149 from Geography
Doing Aegan...
Found 126 Individuals; 126 from Geography
Doing Central Europe...
Found 191 Individuals; 184 from Geography
Doing Black Sea...
Found 32 Individuals; 28 from Geography
Doing North Africa...
Found 56 Individuals; 55 from Geography
Doing Britain...
Found 166 Individuals; 162 from Geography
Doing Baltic Sea...
Found 117 Individuals; 117 from Geography
Doing Sardinia...
Found 76 Individuals; 76 from Geography
Doing Levante...
Found 204 Individuals; 203 from Geography
Doing Eastern Europe...
Found 141 Individuals; 141 from Geography
Doing Central Italy...
Found 135 Individuals; 135 from Geography
Doing Steppe...
Found 553 Individuals; 553 from Geography
Doing Patagonia...
Found 10 Individuals; 10 from Geography
Doing Andean...
Found 39 Individuals; 39 from Geography
Doing Pacific NW...
Found 19 Individuals; 19 from Geography
Doing Atlantic Coast...
Found 22 Indi

## Set the colors

### Set the Economies (Mode of Food Production)

In [33]:
df_t["color"]= "gray" # Make Tabula Rasa
csv_path = "./Data/RegionDefinition/colors.csv"
#df_t = set_colors_from_csv(df_t, csv_path)
#df_t = set_color_hg_minage(df_t, color="purple")

### Set it based on Food Economy
df_t = merge_in_economy_iid(df_t, path_economy="./Data/RegionDefinition/economy_clst_v3.csv", match_col='clst')   # Do the Individual Matches (overwriting)
df_t = merge_in_economy_iid(df_t, path_economy="./Data/RegionDefinition/economy_iid_v3.csv", match_col='iid')   # Do the Individual Matches (overwriting)
df_t = change_economy(df_t, col='economy', str1='Agriculture mixed', new='Agriculture', output=True)
df_t = change_economy(df_t, col='economy', str1='Pastoralism mixed', new='Pastoralism', output=True)
df_t = change_economy(df_t, col='economy', str1='Foraging mixed', new='Foraging', output=True)
df_t = set_economy_color(df_t, path_color_df="./Data/RegionDefinition/economy_colors_v3.csv")
df_t = set_color_modern(df_t, color="gainsboro")

Changing 460 Individuals to: Agriculture
Changing 5 Individuals to: Pastoralism
Changing 23 Individuals to: Foraging
Found 1941 Moderns - set to color: gainsboro


### Save the Summary Dataframe

In [34]:
savepath="./Empirical/roh_all_inds_final_v42.1.csv"
if len(savepath)>0:
    df_t.to_csv(savepath, sep="\t", index=False)
    print(f"Saved {len(df_all)} Individual ROH to: {savepath}")

Saved 3726 Individual ROH to: ./Empirical/roh_all_inds_final_v42.1.csv


# Area 51

In [8]:
df = pd.read_csv("./Empirical/roh_all_inds_final_v42.1.csv", sep="\t")

In [11]:
len(df[df["age"]>0])

1785

In [35]:
df_t["economy"].value_counts()

Agriculture        1119
Pastoralism         345
Foraging            240
Low Certainty        66
Aceramic Farmer      13
Not Assigned          1
Name: economy, dtype: int64

In [19]:
df_t[df_t["clst"]=="Russia_Bolshoy"]

,iid,pop,max_roh,sum_roh>4,n_roh>4,sum_roh>8,n_roh>8,sum_roh>12,n_roh>12,sum_roh>20,...,lon,age,study,clst,mean_cov,n_cov_snp,include_alt,region,color,economy
iid,,,,,,,,,,,,,,,,,,,,,
BOO004.A0101,BOO004.A0101,Russia_Bolshoy,32.012301,153.249483,15,94.091290,5,74.916696,3,58.141296,...,36.366667,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,2.730692,777001,1,Islands,blue,Agriculture
BOO005.A0101,BOO005.A0101,Russia_Bolshoy,23.379206,145.998818,14,102.499722,7,73.517417,4,44.828009,...,36.366667,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,2.690273,717504,1,Islands,blue,Agriculture
BOO001.A0101,BOO001.A0101,Russia_Bolshoy,35.236603,180.272104,17,131.149401,8,102.846979,5,35.236603,...,36.366667,3745.0,LamnidisNatureCommunications2018,Russia_Bolshoy,1.126532,615153,1,Islands,blue,Agriculture
BOO003.A0101,BOO003.A0101,Russia_Bolshoy,18.153601,181.349129,23,92.981807,8,43.635599,3,0.000000,...,36.366667,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,1.006430,609521,1,Islands,blue,Agriculture
BOO006.A0101,BOO006.A0101,Russia_Bolshoy,26.702786,113.137105,9,95.024791,5,76.246792,3,76.246792,...,36.366667,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,0.995680,602052,1,Islands,blue,Agriculture
BOO002.A0101,BOO002.A0101,Russia_Bolshoy,13.262600,116.166475,16,63.287080,6,26.148099,2,0.000000,...,36.366667,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,0.750120,537219,1,Islands,blue,Agriculture


In [4]:
df_ho = pd.read_csv("./Empirical/HO/combined_roh05.csv", sep="\t")

In [ ]:
df_ho[df_ho["iid"].str.contains("Ju_hoan_North")]

In [ ]:
df_ho[df_ho["pop"].str.contains("Hadza")]